# Goal

Using ML algorithms such as transfer learing and Open CV Computer Vision to identify the objects in the video and build KPI metrics

## Steps:
1. Install and Import Dependencies
2. Load the model
3. Once we achieve the accuracy and performance - Create a docker which can be deployed to cloud platform - AWS, Azure or Google
4. Save it to the database
5. Create a dashboard with KPI metrics

## 1. Install and Import Dependencies

In [1]:
#!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

/home/srinu/video-analytics/yolov5
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-51-g9cd89b7 torch 1.11.0 CPU


Setup complete ✅ (12 CPUs, 62.5 GB RAM, 609.5/1828.6 GB disk)


In [3]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

# 2. Validate

## COCO Test

In [4]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [4]:
# Run YOLOv5x on COCO val
!python3 val.py --weights yolov5s.pt --data coco.yaml --img 640 --iou 0.65 --half

val: data=/home/srinu/video-analytics/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-51-g9cd89b7 torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
val: Scanning '/home/srinu/video-analytics/datasets/coco/val2017.cache' images a
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.651      0.525      0.561      0.361
Speed: 0.7ms pre-process, 77.5ms inference, 5.9ms NMS per image at shape (32, 3, 640, 640)

Evaluating pycocotools mAP... saving runs/val/exp4/yolov5s_predictions.json...
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Loading and preparin

# 3. Train

In [5]:
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [6]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

Note: you may need to restart the kernel to use updated packages.


wandb: Currently logged in as: srikaku (use `wandb login --relogin` to force relogin)


True

In [7]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 10 --data coco128.yaml --weights yolov5s.pt --cache

wandb: Currently logged in as: srikaku (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-51-g9cd89b7 torch 1.11.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        929       0.82      0.681      0.789      0.518

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       7/9        0G   0.04381   0.05817   0.01211       323       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        929      0.811      0.705      0.812      0.537

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       8/9        0G   0.04195   0.05843   0.01319       261       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        929      0.767      0.755      0.808      0.551

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       9/9        0G   0.04086   0.05376   0.01117       190       640: 100%|███
               Class     Images     L

Exception ignored in: <function StorageWeakRef.__del__ at 0x7fd355ec7940>
Traceback (most recent call last):
  File "/home/srinu/anaconda3/lib/python3.9/site-packages/torch/multiprocessing/reductions.py", line 36, in __del__
  File "/home/srinu/anaconda3/lib/python3.9/site-packages/torch/storage.py", line 520, in _free_weak_ref
AttributeError: 'NoneType' object has no attribute '_free_weak_ref'
Exception ignored in: <function StorageWeakRef.__del__ at 0x7fd355ec7940>
Traceback (most recent call last):
  File "/home/srinu/anaconda3/lib/python3.9/site-packages/torch/multiprocessing/reductions.py", line 36, in __del__
  File "/home/srinu/anaconda3/lib/python3.9/site-packages/torch/storage.py", line 520, in _free_weak_ref
AttributeError: 'NoneType' object has no attribute '_free_weak_ref'
Exception ignored in: <function StorageWeakRef.__del__ at 0x7fd355ec7940>
Traceback (most recent call last):
  File "/home/srinu/anaconda3/lib/python3.9/site-packages/torch/multiprocessing/reductions.py",

# 4. Visualize

In [8]:
from utils.plots import plot_results 
plot_results('runs/train/exp/results.csv')  # plot 'results.csv' as 'results.png'

# 5. Test

In [9]:
# Detect only people
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --classes 0 --source ../data/VIRAT_S_010002_01_000123_000148.mp4

detect: weights=['yolov5s.pt'], source=../data/VIRAT_S_010002_01_000123_000148.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-51-g9cd89b7 torch 1.11.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.067s)
video 1/1 (2/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (3/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (4/548) /home/srinu/video-analytics/data/V

video 1/1 (63/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 3 persons, Done. (0.046s)
video 1/1 (64/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.045s)
video 1/1 (65/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.046s)
video 1/1 (66/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.045s)
video 1/1 (67/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.041s)
video 1/1 (68/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.044s)
video 1/1 (69/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.043s)
video 1/1 (70/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.043s)
video 1/1 (71/54

video 1/1 (131/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.056s)
video 1/1 (132/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.059s)
video 1/1 (133/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.056s)
video 1/1 (134/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.059s)
video 1/1 (135/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.058s)
video 1/1 (136/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.060s)
video 1/1 (137/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.060s)
video 1/1 (138/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.057s)
video 1/1 (139/548) /home/

video 1/1 (201/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (202/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (203/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (204/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.049s)
video 1/1 (205/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.052s)
video 1/1 (206/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.049s)
video 1/1 (207/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.053s)
video 1/1 (208/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.057s)
video 1/1 (209/548) /home/srinu/video-analytics/data/VIR

video 1/1 (271/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.053s)
video 1/1 (272/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.049s)
video 1/1 (273/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.050s)
video 1/1 (274/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.051s)
video 1/1 (275/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.051s)
video 1/1 (276/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.051s)
video 1/1 (277/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.049s)
video 1/1 (278/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.049s)
video 1/1 

video 1/1 (339/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (340/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.055s)
video 1/1 (341/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (342/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.047s)
video 1/1 (343/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.049s)
video 1/1 (344/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.048s)
video 1/1 (345/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.049s)
video 1/1 (346/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (347/548) /home/

video 1/1 (408/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.047s)
video 1/1 (409/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.052s)
video 1/1 (410/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.048s)
video 1/1 (411/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.048s)
video 1/1 (412/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.051s)
video 1/1 (413/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.048s)
video 1/1 (414/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.047s)
video 1/1 (415/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.047s)
video 1/1 (416

video 1/1 (475/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.051s)
video 1/1 (476/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.046s)
video 1/1 (477/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.048s)
video 1/1 (478/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.049s)
video 1/1 (479/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.050s)
video 1/1 (480/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.043s)
video 1/1 (481/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.058s)
video 1/1 (482/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.054s)
video 1/1 (483/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x64

video 1/1 (545/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.049s)
video 1/1 (546/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.049s)
video 1/1 (547/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.046s)
video 1/1 (548/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.045s)
Speed: 0.4ms pre-process, 50.4ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp
